# Segment, localise and track


In [1]:
from macrohet import dataio, tile, notify
import numpy as np
from tqdm.auto import tqdm
from cellpose import models
import btrack 
import torch
import os
import dask.array as da
import glob
import zarr
import logging
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

# # defining personal trained cellpose model to use
# model_path = '/home/dayn/analysis/models/cellpose/PS0000/macrohet_seg'
# model = models.CellposeModel(gpu=True, 
#                              pretrained_model=model_path)

# ORRRR test the new cellpose model
model = models.Cellpose(gpu=True, model_type='cyto3')

# Initialize the logging configuration
log_dir = "logs"  # Specify the directory where logs will be saved
os.makedirs(log_dir, exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logging.getLogger('cellpose').setLevel(logging.WARNING)

# Add a FileHandler to save logs to a file in the specified directory
log_file = os.path.join(log_dir, "assay_processing.log")
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s [%(levelname)s]: %(message)s")
file_handler.setFormatter(formatter)
logging.getLogger().addHandler(file_handler)

# Define a function to log progress and potential errors
def log_progress(position, message):
    logging.info(f"Position {position}: {message}")


Using device: cuda

NVIDIA RTX A6000
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:>>>> using GPU
INFO:cellpose.models:>> cyto3 << model set to be used
INFO:cellpose.models:>>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


### Define functions to tidy up main block of code

In [2]:
# define thresholds
segment_size_thresh = 5000
Mtb_load_thresh = 480

# define tracking scale factor
scale_factor = 1/5.04

# define features to use for tracking 
features = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "mean_intensity",
    ]

# define tracker config fn to use, using a prob_not_assign = 0.1
config_fn = '/home/dayn/analysis/models/btrack/particle_config_pnassign.json'
# define tracker config fn to use
# config_fn = '/home/dayn/analysis/btrack/models/particle_config.json'

def segment(frame, model = model, channels = [0,0], diameter = 350, #250 #325
            min_size = 5000, model_type = 'pretrained'
           ):
    
    if model_type == 'pretrained':
        
        masks, flows, styles, diams = model.eval(frame, # for default models
                                                 channels = channels, 
                                                 diameter = diameter, 
                                                 min_size = min_size, 
                                                 )
        
        
    else:

        masks, flows, styles = model.eval(frame, # for personal model
                                          channels = channels, 
                                          diameter = diameter, 
                                          min_size = min_size, 
                                          )
    return masks


def localise(masks, intensity_image, properties=tuple(features), use_weighted_centroid = False):
    
    # localise objs in images
    objects = btrack.utils.segmentation_to_objects(segmentation=masks,
                                                   intensity_image=intensity_image, 
                                                   properties=properties,
                                                   scale=(scale_factor,scale_factor),
                                                   use_weighted_centroid=use_weighted_centroid, 
                                                   )
                                                   
    return objects


def track(objects, masks, config_fn, search_radius = 20):

    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure(config_fn)
        # set max search radius
        tracker.max_search_radius = search_radius
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # redefine features so that both channels are included in track measurements
        tracker.features = list(objects[0].properties.keys())
        # append the objects to be tracked
        tracker.append(objects)
        # set the tracking volume
        tracker.volume=((0, masks.shape[-2]*scale_factor), (0, masks.shape[-1]*scale_factor))
        # track them (in interactive mode)
        tracker.track(step_size=25)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # store the tracks
        tracks = tracker.tracks

    return tracks


def save_masks(file_path, masks, overwrite=False):
    with btrack.io.HDF5FileHandler(file_path, 'w', obj_type='obj_type_1') as writer:
        writer.write_segmentation(masks)

def load_masks(file_path):
    if os.path.exists(file_path):
        with btrack.io.HDF5FileHandler(file_path, 'r') as reader:
            return reader.segmentation
    return []


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [3]:
from skimage import io

In [5]:
base_dir = '/mnt/SYNO/macrohet_syno/data/ND0004/'
io.imread(os.path.join(base_dir, 'acquisition/Images/r03c01f01p01-ch1sk1fk1fl1.tiff')).shape

(2160, 2160)

In [4]:
base_dir = '/mnt/SYNO/macrohet_syno/data/ND0004/'
metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0301K1F1P1R1,Ok,r03c01f01p01-ch1sk1fk1fl1.tiff,3,1,1,1,0,1,1,...,0,0.135243401,0,2024-04-17T15:13:28.903+01:00,640,706,40,1.1,0.2,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
1,0301K1F1P1R2,Ok,r03c01f01p01-ch2sk1fk1fl1.tiff,3,1,1,1,0,2,1,...,0,0.135243401,0,2024-04-17T15:13:29.123+01:00,488,522,40,1.1,0.1,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
2,0301K1F1P2R1,Ok,r03c01f01p02-ch1sk1fk1fl1.tiff,3,1,1,2,0,1,1,...,2E-06,0.135245398,0,2024-04-17T15:13:29.467+01:00,640,706,40,1.1,0.2,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
3,0301K1F1P2R2,Ok,r03c01f01p02-ch2sk1fk1fl1.tiff,3,1,1,2,0,2,1,...,2E-06,0.135245398,0,2024-04-17T15:13:29.7+01:00,488,522,40,1.1,0.1,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
4,0301K1F1P3R1,Ok,r03c01f01p03-ch1sk1fk1fl1.tiff,3,1,1,3,0,1,1,...,4E-06,0.135247394,0,2024-04-17T15:13:30.043+01:00,640,706,40,1.1,0.2,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349267,0612K154F9P1R2,Ok,r06c12f09p01-ch2sk154fk1fl1.tiff,6,12,9,1,153,2,1,...,0,0.1350099,275404.583,2024-04-20T19:57:46.98+01:00,488,522,40,1.1,0.1,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
349268,0612K154F9P2R1,Ok,r06c12f09p02-ch1sk154fk1fl1.tiff,6,12,9,2,153,1,1,...,2E-06,0.135011896,275404.583,2024-04-20T19:57:47.323+01:00,640,706,40,1.1,0.2,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
349269,0612K154F9P2R2,Ok,r06c12f09p02-ch2sk154fk1fl1.tiff,6,12,9,2,153,2,1,...,2E-06,0.135011896,275404.583,2024-04-20T19:57:47.557+01:00,488,522,40,1.1,0.1,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
349270,0612K154F9P3R1,Ok,r06c12f09p03-ch1sk154fk1fl1.tiff,6,12,9,3,153,1,1,...,4E-06,0.135013893,275404.583,2024-04-20T19:57:47.9+01:00,640,706,40,1.1,0.2,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [5]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC  Replicate #
Row Column                                                           
3   1         UNI     CTRL             0             EC0            1
    2         UNI     CTRL             0             EC0            2
    3          WT     CTRL             0             EC0            1
    4          WT     CTRL             0             EC0            2
    5          WT      PZA            60            EC50            1
    6          WT      PZA            60            EC50            2
    7          WT      RIF           0.1            EC50            1
    8          WT      RIF           0.1            EC50            2
    9          WT      INH          0.04            EC50            1
    10         WT      INH          0.04            EC50            2
    11         WT      BDQ          0.02            EC50            1
    12         WT      BDQ          0.02            EC50            2
4   3          WT     CTRL             0             EC0            3
    4          WT     CTRL             0             EC0            4
    5          WT      PZA           400            EC99            1
    6          WT      PZA           400            EC99            2
    7          WT      RIF             2            EC99            1
    8          WT      RIF             2            EC99            2
    9          WT      INH             2            EC99            1
    10         WT      INH             2            EC99            2
    11         WT      BDQ           2.5            EC99            1
    12         WT      BDQ           2.5            EC99            2
5   3         RD1     CTRL             0             EC0            1
    4         RD1     CTRL             0             EC0            2
    5         RD1      PZA            60            EC50            1
    6         RD1      PZA            60            EC50            2
    7         RD1      RIF           0.1            EC50            1
    8         RD1      RIF           0.1            EC50            2
    9         RD1      INH          0.04            EC50            1
    10        RD1      INH          0.04            EC50            2
    11        RD1      BDQ          0.02            EC50            1
    12        RD1      BDQ          0.02            EC50            2
6   3         RD1     CTRL             0             EC0            3
    4         RD1     CTRL             0             EC0            4
    5         RD1      PZA           400            EC99            1
    6         RD1      PZA           400            EC99            2
    7         RD1      RIF             2            EC99            1
    8         RD1      RIF             2            EC99            2
    9         RD1      INH             2            EC99            1
    10        RD1      INH             2            EC99            2
    11        RD1      BDQ           2.5            EC99            1
    12        RD1      BDQ           2.5            EC99            2

# Segment, localise and track

In [6]:
mtb_channel = 0
gfp_channel = 1
manual_mtb_thresh_channel = 2
output_dirname = 'cpv3' #'cpv3_smaller_diam' #'cpv3'
# make output directory and subdirs 
os.makedirs(os.path.join(base_dir, f'labels/{output_dirname}/backup'), exist_ok = True)

# Iterate over all positions

In [7]:
# Inside your loop, use the log_progress function to log progress and errors
for (row, column), info in tqdm(assay_layout.iterrows(), desc='Progress through positions', total=len(assay_layout)):
    try:
        acq_ID = (row, column)
        log_progress(acq_ID, "Starting new acquisition")
        
        # if info['Strain'] == 'UNI':
        #     log_progress(acq_ID, "Skipping uninfected acquisition for now")
        #     continue
        # if acq_ID in already_processed_acq_IDs:
        #     log_progress(acq_ID, "Skipping already processed")
        #     continue
        if os.path.exists(os.path.join(base_dir, f'labels/{output_dirname}/{row, column}.h5')):
            log_progress(acq_ID, "Skipping already processed")
            continue

        # process images using zarr
        image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
        zarr_store = zarr.open(image_dir, mode='r')
        images = zarr_store.images
        # create a max projection
        images = np.max(images, axis = 2)
        
        log_progress(acq_ID, "Images loaded and stacked")
        
        # check if already segmented using m2 model
        #     if os.path.exists(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}.h5')):
        #         continue
        #     else:
        log_progress(acq_ID, "Starting segmentation")
        
        #     if os.path.exists(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}_first_pass_seg_backup.h5')):

        # # Define the path for saving masks
        # output_file_path = os.path.join(base_dir, f'labels/{output_dirname}/{row, column}_cpv3_mask_backup.h5')
        
        # # Load existing masks
        # masks = load_masks(output_file_path)
        
        # # Check if the required number of masks are already present
        # if len(masks) < len(images):
        #     for i, frame in enumerate(tqdm(images[:, gfp_channel, ...], desc='Segmenting')):
        #         if i >= len(masks):
        #             mask = segment(frame, model)
        #             masks.append(mask)
        #             save_masks(output_file_path, np.array(masks), overwrite=True)
        #     log_progress(acq_ID, "Finished segmentation")
        # else:
        #     log_progress(acq_ID, f"Skipped segmentation, found {len(masks)} masks")

        
        # segment images from gfp channel only
        masks = np.stack([segment(frame) 
                          for frame in tqdm(images[:,gfp_channel,...],  # segmenting the GFP channel 
                                            desc = 'Segmenting')])
        
        log_progress(acq_ID, "Finished segmentation")
        
        with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/{row, column}_cpv3_mask_backup.h5'), 
                                           'w', 
                                           obj_type='obj_type_1'
                                           ) as writer:
        #             writer.write_objects(objects)
                # writer.write_tracks(tracks)
                writer.write_segmentation(masks)
            
        log_progress(acq_ID, "Saved out masks")  
        
        log_progress(acq_ID, "Measuring Mtb area")       
        
        # characterise Mtb growth using Otsu segmentation
        # otsu_mtb = otsu_threshold(images[:,1,...]) # time consuming and non-deterministic when compared to hardcoded, could result in different thresholds for same image? 
        # characterise Mtb growth using hardcoded threshold :S
        manual_mtb_thresh = np.where(images[:,mtb_channel,...] >= Mtb_load_thresh, True, False)
        log_progress(acq_ID, "Creating intensity image for localisation")  
        # reshape intensity image to be gfp, rfp on last axis for regionprops
        intensity_image = np.stack([images[:,0,...], 
                                    images[:,1,...],  
        #                                 otsu_mtb, 
                                    manual_mtb_thresh], axis = -1)
        log_progress(acq_ID, "Localising objects")  
        # localise objects
        objects = localise(masks, 
                           intensity_image, 
                           )
        log_progress(acq_ID, "Filtering small objects")  
        # filter out small objects
        objects = [o for o in objects if o.properties['area'] > segment_size_thresh]
        
        log_progress(acq_ID, "Adding infection labels to objects")  
        # add label for infection
        for obj in objects:
            obj.properties = ({"Infected": True} 
                                if obj.properties['mean_intensity'][manual_mtb_thresh_channel] > 0 # index 2 for manual mtb channel 
                                else {"Infected": False})
            obj.properties = ({"Mtb area px": obj.properties['mean_intensity'][manual_mtb_thresh_channel]*obj.properties['area']})
        
        with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/backup/{row, column}_cpv3_objects_backup.h5'), 
                                                   'w', 
                                                   obj_type='obj_type_1'
                                                   ) as writer:
                        writer.write_objects(objects)
                        # writer.write_tracks(tracks)
        
        log_progress(acq_ID, "Beginning tracking")  
        # track on upscaled config fn
        tracks = track(objects, masks, config_fn, search_radius = 20)
        log_progress(acq_ID, "Saving tracking")  
        # save out 
        with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/backup/{row, column}_cpv3_tracks_backup.h5'), 
                                           'w', 
                                           obj_type='obj_type_1'
                                           ) as writer:
        #             writer.write_objects(objects)
                writer.write_tracks(tracks)
                # writer.write_segmentation(masks)
        # Log successful completion
        
        with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/{row, column}.h5'), 
                                           'w', 
                                           obj_type='obj_type_1'
                                           ) as writer:
                    writer.write_tracks(tracks)
                    writer.write_segmentation(masks)
        # Log successful completion
        log_progress(acq_ID, "Processing completed successfully")

    except Exception as e:
        # Log errors
        log_progress(acq_ID, f"Processing failed: {str(e)}")

# You can also log information before and after the loop
logging.info("Processing completed")

# Notify if required
notify.send_sms("Processing completed")


Progress through positions:   0%|          | 0/42 [00:00<?, ?it/s]

INFO:root:Position (3, 1): Starting new acquisition
INFO:root:Position (3, 1): Skipping already processed
INFO:root:Position (3, 2): Starting new acquisition
INFO:root:Position (3, 2): Skipping already processed
INFO:root:Position (3, 3): Starting new acquisition
INFO:root:Position (3, 3): Skipping already processed
INFO:root:Position (3, 4): Starting new acquisition
INFO:root:Position (3, 4): Skipping already processed
INFO:root:Position (3, 5): Starting new acquisition
INFO:root:Position (3, 5): Skipping already processed
INFO:root:Position (3, 6): Starting new acquisition
INFO:root:Position (3, 6): Skipping already processed
INFO:root:Position (3, 7): Starting new acquisition
INFO:root:Position (3, 7): Skipping already processed
INFO:root:Position (3, 8): Starting new acquisition
INFO:root:Position (3, 8): Skipping already processed
INFO:root:Position (3, 9): Starting new acquisition
INFO:root:Position (3, 9): Skipping already processed
INFO:root:Position (3, 10): Starting new acqui

Segmenting:   0%|          | 0/154 [00:00<?, ?it/s]

INFO:root:Position (5, 6): Finished segmentation
[INFO][2024/07/29 03:58:04 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(5, 6)_cpv3_mask_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(5, 6)_cpv3_mask_backup.h5...
[INFO][2024/07/29 03:59:00 PM] Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(5, 6)_cpv3_mask_backup.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(5, 6)_cpv3_mask_backup.h5
INFO:root:Position (5, 6): Saved out masks
INFO:root:Position (5, 6): Measuring Mtb area
INFO:root:Position (5, 6): Creating intensity image for localisation
INFO:root:Position (5, 6): Localising objects
[INFO][2024/07/29 03:59:21 PM] Localizing objects from segmentation...
INFO:btrack.io._localization:Localizing objects from segmentation...
[WARNING][2024/07/29 03:59:21 PM] Multichannel intensity image detected, using unweighted centroid.
progress: 100%|██████████████

GLPK Integer Optimizer 5.0
29312 rows, 24996 columns, 35336 non-zeros
24996 integer variables, all of which are binary
Preprocessing...
14656 rows, 24996 columns, 35336 non-zeros
24996 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 14656
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
14656 rows, 24996 columns, 35336 non-zeros
*     0: obj =   8.642744354e+04 inf =   0.000e+00 (6786)
Perturbing LP to avoid stalling [2649]...
*  6769: obj =   4.423810126e+04 inf =   0.000e+00 (79) 1
Removing LP perturbation [6849]...
*  6849: obj =   4.422227937e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  6849: mip =     not found yet >=              -inf        (1; 0)
+  6849: >>>>>   4.422227937e+04 >=   4.422227937e+04   0.0% (1; 0)
+  6849: mip =   4.42

...

Segmenting:   0%|          | 0/154 [00:00<?, ?it/s]

INFO:root:Position (6, 12): Finished segmentation
[INFO][2024/07/31 08:15:55 AM] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(6, 12)_cpv3_mask_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(6, 12)_cpv3_mask_backup.h5...
[INFO][2024/07/31 08:16:48 AM] Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(6, 12)_cpv3_mask_backup.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(6, 12)_cpv3_mask_backup.h5
INFO:root:Position (6, 12): Saved out masks
INFO:root:Position (6, 12): Measuring Mtb area
INFO:root:Position (6, 12): Creating intensity image for localisation
INFO:root:Position (6, 12): Localising objects
[INFO][2024/07/31 08:17:11 AM] Localizing objects from segmentation...
INFO:btrack.io._localization:Localizing objects from segmentation...
[WARNING][2024/07/31 08:17:11 AM] Multichannel intensity image detected, using unweighted centroid.
progress: 100%|█████

GLPK Integer Optimizer 5.0
30076 rows, 25509 columns, 35980 non-zeros
25509 integer variables, all of which are binary
Preprocessing...
15038 rows, 25509 columns, 35980 non-zeros
25509 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 15038
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
15038 rows, 25509 columns, 35980 non-zeros
*     0: obj =   8.830806976e+04 inf =   0.000e+00 (6890)
Perturbing LP to avoid stalling [2536]...
Removing LP perturbation [6986]...
*  6986: obj =   4.645932050e+04 inf =   0.000e+00 (0) 1
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  6986: mip =     not found yet >=              -inf        (1; 0)
+  6986: >>>>>   4.645932050e+04 >=   4.645932050e+04   0.0% (1; 0)
+  6986: mip =   4.645932050e+04 >=     tree is empty   0.0% (0; 1)
INTEGER O

[INFO][2024/07/31 08:23:26 AM] Completed optimization with 5752 tracks
INFO:btrack.core:Completed optimization with 5752 tracks
[INFO][2024/07/31 08:23:27 AM] Ending BayesianTracker session
INFO:btrack.core:Ending BayesianTracker session
INFO:root:Position (6, 12): Saving tracking
[INFO][2024/07/31 08:23:27 AM] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/backup/(6, 12)_cpv3_tracks_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/backup/(6, 12)_cpv3_tracks_backup.h5...
[INFO][2024/07/31 08:23:27 AM] Writing objects/obj_type_1
INFO:btrack.io.hdf:Writing objects/obj_type_1
[INFO][2024/07/31 08:23:27 AM] Writing labels/obj_type_1
INFO:btrack.io.hdf:Writing labels/obj_type_1
[INFO][2024/07/31 08:23:27 AM] Loading objects/obj_type_1 (36214, 5) (36214 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (36214, 5) (36214 filtered: None)
[INFO][2024/07/31 08:23:28 AM] Writing properties/obj_type_1/area (36214,)
INF

Update sms failed to send
